# Trabalho Final da Disciplina de Tópicos Especiais em Matemática Aplicada (FGA0034) 

## Integrantes:
#### 1 - Artur Rodrigues Sousa Alves - 211043638
#### 2 - Danilo Carvalho Antunes - 211039312
#### 3 - Guilherme Soares Rocha - 211039789

## Criação de Agentes para Pesquisar e Escrever um Artigo

Neste trabalho final, o objetivo é criar agentes de inteligência artificial que pesquisam e escrevem um artigo sobre um tema específico. A ferramenta utilizada para a criação dos agente é o [Crew AI](https://www.crewai.com/) que é uma ferramenta que facilita a orquestração e colaboração entre múltiplos agentes de inteligência artificial para resolver tarefas complexas. Ela permite que diferentes agentes, cada um com habilidades e responsabilidades específicas, trabalhem juntos de forma coordenada para atingir um objetivo comum.

🔹 Principais recursos:
- ✅ Coordenação entre múltiplos agentes IA
- ✅ Especialização de agentes em diferentes tarefas
- ✅ Automação de processos complexos
- ✅ Integração com outras ferramentas e APIs

Essa abordagem é útil para aplicações como automação de atendimento ao cliente, análise de dados, desenvolvimento de software e muito mais. 

## - Para instalar as bibliotecas necessárias, execute o comando abaixo:
```Python
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Importação das ferramentas da biblioteca CrewAI para funcionamento do código

In [ ]:
from crewai import Agent, Task, Crew

- Como LLM para os agentes, é utilizado a `Mistral-7B-Instruct-v0.1` através da [Together AI](https://api.together.xyz/).
- Para utilização da SERPER, é necessário a utilização de uma chave de API, que pode ser obtida através do site [SERPER](https://serper.dev/).
- **Obervação:** O Crew AI ainda permite a utilização de outros modelos de LLM para os agentes. Porém decidimos utilizar o modelo `Mistral-7B-Instruct-v0.1` por ser um modelo relativamente robusto e de mais fácil uso.
  
- **Para funcionamento das variáveis abaixo, recomendamos fortemente que se siga os passos acima de modo a obter as chaves necessárias para que o código funcione sem problemas.**


In [ ]:
import os

os.environ["OPENAI_MODEL_NAME"] = "mistralai/Mistral-7B-Instruct-v0.1"
os.environ["SERPER_API_KEY"] = 'COLOQUE AQUI SUA CHAVE SERPER'
os.environ["OPENAI_API_BASE"] = "https://api.together.xyz/v1"
os.environ["OPENAI_API_KEY"] = "COLOQUE AQUI SUA CHAVE GERADA EM TOGETHER AI"

## Criando os Agentes

- Defina seus agentes e dê a eles um `role`, `goal` e `backstory`.
- Foi observado que os LLMs se saem melhor quando estão interpretando um papel.

### Agente: Planejador

**Nota**: O benefício do uso de _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

Contra _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
é capaz de evitar a adição de espaços em branco e caracteres de nova linha, tornando-o melhor formatado para ser passado ao LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agente: Escritor

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agente: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Criando as Tarefas

- Defina suas tarefas e forneça a elas uma `description`, `expected_output` e `agent`.

### Tarefa: Planejamento

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Tarefa: Escrita

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Tarefa: Edição

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Criando a Crew

- Criar a crew de agentes.
- Passar as tarefas a serem realizadas por esses agentes.
    - **Nota**: *Para este exemplo simples*, as tarefas serão realizadas sequencialmente (ou seja, elas são dependentes umas das outras), então a _order_ da tarefa na lista _matters_.
- `verbose=2` permite que você veja todos os logs da execução.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Rodando a Crew

**Nota**: LLMs podem prover diferentes saídas para a mesma entrada, então o que você obtém em uma primeira entrada pode ser diferente de uma outra entrada utilizando o mesmo tema.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

- Mostra os resultados da execução como markdown no notebook.

In [ ]:
# Salvar o resultado em um arquivo Markdown (.md)
md_filename = "result.md"

with open(md_filename, "w", encoding="utf-8") as f:
    f.write(result)

print(f"Arquivo Markdown gerado: {md_filename}")


## Tente você mesmo

- Escreva um tópico da sua escolha e veja como os agentes se saem.

In [ ]:
topic = "COLOQUE AQUI SEU TÓPICO DE INTERESSE"
result = crew.kickoff(inputs={"topic": topic})

- Rode a célula abaixo para gerar um arquivo .md que renderize exatamente o que foi gerado na célula acima.

In [ ]:
# Salvar o resultado em um arquivo Markdown (.md)
md_filename = "your_result.md"

with open(md_filename, "w", encoding="utf-8") as f:
    f.write(result)
